In [2]:
### import
import json
import csv
import pandas as pd
import networkx as nx
import math
from matplotlib import pylab
from matplotlib import pyplot as plt

In [2]:
### single instance example from raw data
# {
#     "attachment":{
#         "type":null,
#         "value":null
#     },
#     "body":"\u041e\u0431\u0435\u0449\u0430\u044e \u0437\u0430\u0431\u043e\u0442\u0438\u0442\u044c\u0441\u044f \u043e \u0442\u0435\u0445, \u043a\u0442\u043e \u0432\u0430\u0436\u0435\u043d \u0434\u043b\u044f \u043c\u0435\u043d\u044f. #\u0412\u041d\u043e\u0432\u043e\u043c\u0413\u043e\u0434\u0443\u042f\u041e\u0431\u0435\u0449\u0430\u044e",
#     "body_html":null,
#     "body_html_summary":null,
#     "body_html_summary_truncated":false,
#     "category":null,
#     "category_details":null,
#     "conversation_parent_id":null,
#     "created_at":"2017-01-01T00:00:01+00:00",
#     "created_utc":1483228801,
#     "dislike_count":0,
#     "edited":false,
#     "embed":{
#         "html":null,
#         "iframe":null
#     },
#     "id":3489400,
#     "is_locked":false,
#     "is_premium":false,
#     "is_quote":false,
#     "is_replies_disabled":false,
#     "is_reply":false,
#     "language":null,
#     "like_count":0,
#     "nsfw":false,
#     "only_emoji":false,
#     "parent_id":null,
#     "quote_conversation_parent_id":null,
#     "reply_count":0,
#     "repost_count":0,
#     "retrieved_utc":1518257233,
#     "revised_at":null,
#     "revised_utc":null,
#     "score":0,
#     "user":{
#         "id":65574,
#         "is_donor":false,
#         "is_investor":false,
#         "is_premium":false,
#         "is_private":false,
#         "is_pro":false,
#         "name":"Dimitri Vladimirovich Metkov",
#         "picture_url":"https://files.gab.ai/user/5835734e4d7dc.jpg",
#         "username":"dima_metkov",
#         "verified":false
#     }
# }

In [3]:
### data raw to data json
list_file_input_raw = [
    'datasets/GABPOSTS_2017-01', 'datasets/GABPOSTS_2017-02', 'datasets/GABPOSTS_2017-03',
    'datasets/GABPOSTS_2017-04', 'datasets/GABPOSTS_2017-05', 'datasets/GABPOSTS_2017-06'
]
file_output_raw_name = "datasets/gab_2017_from_01_to_06.json"
# list_file_input_raw = ['datasets/GABPOSTS_2017-01']
# file_output_raw_name = "datasets/gab_2017_01.json"
list_post = []
for fir in list_file_input_raw:
    print(len(list_post))
    for line in open(fir, 'r'):
        list_post.append(json.loads(line))  
file_output_raw = open(file_output_raw_name, "w")
json.dump(list_post, file_output_raw)
file_output_raw.close()

In [5]:
### data flattening and filter
# file_output_raw = open("datasets/gab_2017_01.json", "r")
file_output_raw = open("datasets/gab_2017_from_01_to_06.json", "r")
list_post= json.load(file_output_raw)
list_flat_post = []
c=0
for p in list_post:
    c+=1
    if c%100000==0:
        print(c/100000)
    e = {}
    e['id'] = p['id'] if 'id' in p.keys() else ''
    e['parent_id'] = p['parent_id'] if 'parent_id' in p.keys() else ''
    e['user__id'] = p['user']['id'] if 'user' in p.keys() and 'id' in p['user'].keys() else ''

    e['body'] = p['body'] if 'body' in p.keys() else ''    
    e['category_details__title'] = p['category_details']['title'] if 'category_details' in p.keys() \
        and p['category_details'] and 'title' in p['category_details'].keys() else ''
    e['category_details__value'] = p['category_details']['value'] if 'category_details' in p.keys() \
        and p['category_details'] and 'value' in p['category_details'].keys() else ''

    e['like_count'] = p['like_count'] if 'like_count' in p.keys() else '' 
    e['dislike_count'] = p['dislike_count'] if 'dislike_count' in p.keys() else '' 
    e['reply_count'] = p['reply_count'] if 'reply_count' in p.keys() else '' 
    e['repost_count'] = p['repost_count'] if 'repost_count' in p.keys() else '' 
    e['is_quote'] = p['is_quote'] if 'is_quote' in p.keys() else '' 
    e['is_reply'] = p['is_reply'] if 'is_reply' in p.keys() else '' 

    e['created_at'] = p['created_at'] if 'created_at' in p.keys() else '' 
    e['created_utc'] = p['created_utc'] if 'created_utc' in p.keys() else '' 
    e['edited'] = p['edited'] if 'edited' in p.keys() else '' 

#     e['attachment__type'] = p['attachment']['type']
#     e['attachment__value'] = p['attachment']['value']    
#     e['body_html'] = p['body_html']
#     e['body_html_summary'] = p['body_html_summary']
#     e['body_html_summary_truncated'] = p['body_html_summary_truncated']
#     e['category'] = p['category']
#     e['category_details__emoji'] = p['category_details']['emoji']
#     e['category_details__slug'] = p['category_details']['slug']
#     e['conversation_parent_id'] = p['conversation_parent_id']
#     e['embed__html'] = p['embed']['html']
#     e['embed__iframe'] = p['embed']['iframe']    
#     e['is_locked'] = p['is_locked']
#     e['is_premium'] = p['is_premium']
#     e['is_replies_disabled'] = p['is_replies_disabled']
#     e['language'] = p['language']
#     e['nsfw'] = p['nsfw']
#     e['only_emoji'] = p['only_emoji']
#     e['quote_conversation_parent_id'] = p['quote_conversation_parent_id']
#     e['retrieved_utc'] = p['retrieved_utc']
#     e['revised_at'] = p['revised_at']
#     e['revised_utc'] = p['revised_utc']
#     e['score'] = p['score']
#     e['user__is_donor'] = p['user']['is_donor']
#     e['user__is_investor'] = p['user']['is_investor']
#     e['user__is_premium'] = p['user']['is_premium']
#     e['user__is_private'] = p['user']['is_private']
#     e['user__is_pro'] = p['user']['is_pro']
#     e['user__picture_url'] = p['user']['picture_url']
#     e['user__username'] = p['user']['username']
#     e['user__name'] = p['user']['name']
#     e['user__verified'] = p['user']['verified']

    list_flat_post.append(e)

In [10]:
### flattened and filtered data to csv
csv_columns = list(list_flat_post[0].keys())
csv_file = "datasets/gab_2017_from_01_to_06.csv"
try:
    with open(csv_file, 'w', encoding="utf-8", newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        c=0
        for fp in list_flat_post:
            writer.writerow(fp)
            c+=1
            if c%100000==0:
                print(c/100000)
except Exception as e:
    print(e)

### large network analysis 

In [3]:
### networkX implementation
df=pd.read_csv('datasets/gab_2017_from_01_to_06.csv', sep=',', header=0) 
m=df.values

In [4]:
### post info
dict_post_info = {}
for r in range(m.shape[0]):
    _id = m[r][0]
    _parent_id = m[r][1] if m[r][1] else ''
    _user_id = m[r][2] if m[r][2] else ''
    dict_post_info[_id] = {
        'parent_id': _parent_id,
        'user_id': _user_id
    }

In [5]:
### user info
dict_user_info = {}
for k,v in dict_post_info.items():
    if v['user_id'] not in dict_user_info.keys():
        dict_user_info[v['user_id']] = []
    if not math.isnan(v['parent_id']) and v['parent_id'] in dict_post_info.keys():
        if dict_post_info[v['parent_id']]['user_id'] not in dict_user_info[v['user_id']]:
            dict_user_info[v['user_id']].append(dict_post_info[v['parent_id']]['user_id'])

In [6]:
### node,edge(child user -> parent user)
list_node = dict_user_info.keys()
list_edge = []
for k,v in dict_user_info.items():
    if len(v) > 0:
        for pu in v:
            edge = (k,pu)
            list_edge.append(edge)

In [7]:
### DiGraph formation
G = nx.DiGraph()
G.add_nodes_from(list_node)
G.add_edges_from(list_edge)

In [12]:
### connected components
nwcc = len(list(nx.weakly_connected_components(G)))
lwcc = G.subgraph(sorted(nx.weakly_connected_components(G), key=len, reverse=True)[0])
nnodes = lwcc.number_of_nodes()
nedges = lwcc.number_of_edges()
print(nwcc)
print(nnodes)
print(nedges)

16740
27083
624690


In [13]:
### clustering coeff
print(nx.average_clustering(G))

0.11584645944034744
